In [1]:
import pandas as pd

In [6]:
# Load all CSV files
df_fact_player = pd.read_csv('../data/Fact_Player.csv')
df_gameweeks = pd.read_csv('../data/Gameweeks.csv')
df_player_history = pd.read_csv('../data/Player_history.csv')
df_players = pd.read_csv('../data/Players.csv')
df_positions = pd.read_csv('../data/Positions.csv')
df_teams = pd.read_csv('../data/Teams.csv')

In [ ]:
print(df_fact_player.head())
print(df_gameweeks.head())
print(df_player_history.head())
print(df_players.head())
print(df_positions.head())
print(df_teams.head())

In [10]:
# Merge fact player data with players, positions, and teams
merged_df = pd.merge(df_fact_player, df_players, left_on='element', right_on='id', how='left')
merged_df = pd.merge(merged_df, df_positions, left_on='element_type', right_on='id', how='left')
merged_df = pd.merge(merged_df, df_teams, left_on='opponent_team', right_on='id', how='left')
merged_df = pd.merge(merged_df, df_gameweeks, left_on='GW', right_on='id', how='left', suffixes=('_left', '_right'))
# Rename columns for clarity
merged_df = merged_df.rename(columns={
    'team_name': 'opponent_team_name',
    'singular_name': 'position',
    'GW': 'gameweek'
})

# Preview the merged dataframe
print(merged_df.head())

MergeError: Passing 'suffixes' which cause duplicate columns {'id_x'} is not allowed.